![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)



In [ ]:
%%html
<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }
  
  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>

## Phidgets_6

Using what we have learned so far, let's add some graphs to the display. This way we can see record over time of the temperature and humidity readings. 


## Setup the device

Click the button below "Connect Phidgets" to connect your phidget device to this Jupyter notebook.

You must be running on Chrome on your laptop or desktop, and the Phidget device needs to be connected to the USB port on your computer. 

The Javascript code here is much like before, except now we have introduced callback functions "openHumidityPhidget" and "openTemperaturePhidget." A callback function is just a function that gets called whenever these JS function operate.

What is nice here is the callback function is written in Python, so it is easy for us to program them in Python, and make changes as we need to. 

In [1]:
%%html
<script>
requirejs.config({
    paths: { 
        'phidget22': ['https://unpkg.com/phidget22@^3.10/browser/phidget22'], 
    },                                  
});

var usbconn;
require(['phidget22'], function(phidget22) {
    console.log(phidget22);  
    if(usbconn == null){
        usbconn = new phidget22.USBConnection({
        name: "USB Devices",
        onError: (code, msg) => { console.error("Connection Error:", msg); }
        });
    }
    console.log(usbconn)
    
    usbconn.connect().catch(err => {
        usbconn.delete();
        console.error("Error connecting to USB", err);
        alert('Failed to connect to USB: ' + err);
    });
    
    openHumidityPhidget();
    openTemperaturePhidget();
    
    return {};
});

function openHumidityPhidget(){
    require(['phidget22'], function(phidget22) {
        let humiditySensor = new phidget22.HumiditySensor();
        humiditySensor.onHumidityChange = function (humidity) {
            IPython.notebook.kernel.execute(
                "callback_hum(" + humidity + ");"
            );  
        };
        humiditySensor.open();
        return {};
});
}

function openTemperaturePhidget(){
    require(['phidget22'], function(phidget22) {
        let temperatureSensor = new phidget22.TemperatureSensor();
        temperatureSensor.onTemperatureChange = function (temperature) {
            IPython.notebook.kernel.execute(
                "callback_temp(" + temperature + ");"
            );  
        };
        temperatureSensor.open();
        return {};
});
}
    
function connectPhidgets(){
    require(['phidget22'], function(phidget22) {
    console.log(usbconn);  // or whatever
    try {
        usbconn.requestWebUSBDeviceAccess();
    } catch (err) {
        console.error('Request device error', err);
    };
    return {};
});
}

</script>
<div>
<button onclick="connectPhidgets()">Connect Phidgets</button>
</div>

### Libraries

We include a few libraries, for plotting and doing numerical work.

In [2]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np

## Utility functions

We introduce a few utility functions, which are used to set the ranges on the Y axis of our graphs. We don't know what temperature or humidity to expect, so we set the Y range to about $\pm 2$ degrees of the current temperature value, and $\pm 10$ percentage points for the humidity. 

This will make it easier for us to see small changes in the reading, reflected in the graphs.

In [3]:
## Utility function to bracket a value by plus/minus 2, 5 or 10

def bracket2(value):
    return (2*np.round(value/2)-2, 2*np.round(value/2)+2)

def bracket5(value):
    return (5*np.round(value/5)-5, 5*np.round(value/5)+5)

def bracket10(value):
    return (10*np.round(value/10)-10, 10*np.round(value/10)+10)

## Callback functions

Here we define the callback functions. These functions get called whenevern the Phidgets report new values for temperature and humidity. They are initiatied from the Javascript code above.

We have two buffers, or arrays, to store the temperature and humidity readings over a period of time. We chose 100 points, for simplicity, but you could change this. 

The callback functions do three things:
- update the value in the temperature or humidity gauge
- store the new value at the end of buffer, and display the 100 values in the temperature or humidity graph
- update the range of the y-axis for the graph, so it brackets the actual reading value.

As mentioned above, we do this bracketing so the graphs look nice, and we can see small changes in tmeperature or humidity.

Feel free to experiment on better ways to display the charts. 

In [4]:
buffer_len = 100

h_buffer = np.zeros(buffer_len)
t_buffer = np.zeros(buffer_len)

def callback_temp(value):
    f.data[0]['value'] = value      
    t_buffer[0:-1] = t_buffer[1:]
    t_buffer[-1] = value
    f.data[2]['y'] = t_buffer
    f.layout['yaxis']['range'] = bracket2(value)
   
def callback_hum(value):
    f.data[1]['value'] = value
    h_buffer[0:-1] = h_buffer[1:]
    h_buffer[-1] = value
    f.data[3]['y'] = h_buffer
    f.layout['yaxis2']['range'] = bracket10(value)
 

## The gauges and charts

Finally, we create the two gauges for temperature and humidity, and the two charts to display the data over time. The gauges are called **t_indicator** and **h_indicator**, and the charts are called **t_chart** and **h_chart**.

We use the features of Plotly to put this four items into a single figure, using the **subplot()** function. We also add titles and some default ranges for the y-axes in the charts, to make this all work with the callback functions above. 

We then turn the figure into a live widget, using the **FigureWidget()** function

In [5]:
x = np.arange(buffer_len)

t_indicator = go.Indicator(
    mode = "gauge+number",
    value = 20,
    domain = {'x': [0, 1], 'y': [0, 1]},
    title = {'text': "Temperature"},
    gauge = {'axis': {'range': [5, 35]}}
)

h_indicator = go.Indicator(
    mode = "gauge+number",
    value = 40,
    domain = {'x': [0, 1], 'y': [0, 1]},
    title = {'text': "Humidity"},
    gauge = {'axis': {'range': [0, 100]}}
)


t_chart = go.Scatter(x=x, y=t_buffer, mode='lines')
h_chart = go.Scatter(x=x, y=h_buffer, mode='lines')


fig = make_subplots(
    rows=2,
    cols=2,
    specs=[[{'type' : 'domain'}, {'type' : 'domain'}],
           [{'type' : 'xy'}, {'type' : 'xy'}]])
fig.append_trace(t_indicator, row=1, col=1)
fig.append_trace(h_indicator, row=1, col=2)
fig.append_trace(t_chart, row=2, col=1)
fig.append_trace(h_chart, row=2, col=2)

fig.update_traces(name='Temperature', showlegend = True, row=2, col=1)
fig.update_traces(name='Humidity', showlegend = True, row=2, col=2)

fig.update_yaxes(title_text="Degrees C", range=[10, 30], row=2, col=1)
fig.update_yaxes(title_text="Percentage", range=[50, 70], row=2, col=2)

f = go.FigureWidget(fig)

f

FigureWidget({
    'data': [{'domain': {'x': [0.0, 0.45], 'y': [0.575, 1.0]},
              'gauge': {'axis': …

## Going further

Some ideas to try on your own:

- is there a better way to bracket the values when we set the y-axis limits? Can we make the display nicer?
- can we add some "memory" to the y-axis ranges, so they don't jump around too much when the temperature or humidity reading crosses some critical points? For instance, when the temperature crosses an odd integer (e.g. value = 27), the range will jump around from (24,28) t0 (26,30). Which is a rather ugly display
- can we update the charts only once a second? That way it is easier to see the slow changes in value.
- can we record the x-axis as a time value, so we know at what time certain readings occur?
- can we save the data into a file, on some regular basis? This should include data and time stamps, so we have an accurate historical record of the reading.


[![Callysto.ca License](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-bottom.jpg?raw=true)](https://github.com/callysto/curriculum-notebooks/blob/master/LICENSE.md)